In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Inference_API_KEY_8B` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from transformers import pipeline
from PIL import Image
from google.colab.patches import cv2_imshow

# Load your trained YOLO model
model_path = r"/content/best.pt"
model = YOLO(model_path)

# Video path and capture
video_path = r"/content/Test_video_1.mp4"
cap = cv2.VideoCapture(video_path)

# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
middle_index = total_frames // 2 if total_frames > 0 else -1
selected_frame = None

# Dictionaries to store vehicle counts and track mappings
vehicle_counts = {}  # Counts unique vehicles per type
track_id_to_class = {}  # Maps track IDs to vehicle types

frame_index = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Optionally resize the frame
    frame = cv2.resize(frame, (700, 500))

    # Run YOLO tracking
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes in [x1, y1, x2, y2]
    confidences = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy().astype(int)
    # Extract track IDs, handle case where no IDs are present
    track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

    # Count unique vehicles
    for conf, cls, track_id in zip(confidences, classes, track_ids):
        if conf < 0.5:
            continue  # Filter low-confidence detections
        label = results[0].names[cls]  # Get vehicle type (e.g., "car", "ambulance")
        if track_id not in track_id_to_class:
            # New vehicle detected
            track_id_to_class[track_id] = label
            vehicle_counts[label] = vehicle_counts.get(label, 0) + 1

    # Visualize predictions (optional, includes track IDs)
    annotated_frame = results[0].plot()

    # Save the middle frame for the report
    if frame_index == middle_index:
        selected_frame = frame.copy()

    frame_index += 1

cap.release()
cv2.destroyAllWindows()

# Convert selected frame to PIL image
if selected_frame is not None:
    selected_frame_rgb = cv2.cvtColor(selected_frame, cv2.COLOR_BGR2RGB)
    selected_frame_pil = Image.fromarray(selected_frame_rgb)
else:
    print("Warning: No frame was selected. Using counts only for report generation.")
    selected_frame_pil = None

# Set up the image-text-to-text pipeline with Llama-3.2-11B-Vision
pipe = pipeline("image-text-to-text", model="meta-llama/Llama-3.2-11B-Vision", use_auth_token=True)

# Build the report prompt with unique vehicle counts
report_prompt = (
    f"Based on this frame from a vehicle detection video and the following unique vehicle counts across all frames:\n"
)
for vehicle_type, count in vehicle_counts.items():
    report_prompt += f"{vehicle_type}: {count}\n"
report_prompt += "Generate a report summarizing the unique vehicle counts and describing the scene in the selected frame."

# Generate the initial report
if selected_frame_pil is not None:
    result = pipe(image=selected_frame_pil, text=report_prompt, max_new_tokens=300)
    report_text = result[0]['generated_text']
else:
    # Fallback prompt without image
    report_prompt_fallback = (
        f"Vehicle Detection Report:\n"
    )
    for vehicle_type, count in vehicle_counts.items():
        report_prompt_fallback += f"{vehicle_type}: {count}\n"
    report_prompt_fallback += "Based on these unique counts, provide a summary of the vehicle detections."
    result = pipe(image=None, text=report_prompt_fallback, max_new_tokens=300)
    report_text = result[0]['generated_text']

# Print the generated report
print("\n--- Generated Report ---\n")
print(report_text)

# Start chat interaction
print("\nVideo processing complete. You can now ask questions about the video (type 'exit' to quit).")

while True:
    user_query = input("Ask a question: ")
    if user_query.lower() == "exit":
        print("Goodbye.")
        break

    # Handle specific counting questions
    if "how many vehicles" in user_query.lower() or "total number of vehicles" in user_query.lower():
        total_vehicles = sum(vehicle_counts.values())
        response = f"There are {total_vehicles} vehicles in total in the video."
    elif "how many" in user_query.lower():
        words = user_query.lower().split()
        vehicle_type = None
        for word in words:
            if word in vehicle_counts:
                vehicle_type = word
                break
            elif word + "s" in vehicle_counts:  # Check plural by adding 's'
                vehicle_type = word + "s"
                break
            elif word.endswith("s") and word[:-1] in vehicle_counts:  # Check singular by removing 's'
                vehicle_type = word[:-1]
                break
        if vehicle_type and vehicle_type in vehicle_counts:
            count = vehicle_counts[vehicle_type]
            response = f"There are {count} {vehicle_type}(s) in the video."
        else:
            response = "I'm not sure about that vehicle type."
    else:
        # Build prompt for general questions
        prompt = f"Based on the following unique vehicle counts across all frames:\n"
        for vt, count in vehicle_counts.items():
            prompt += f"{vt}: {count}\n"
        prompt += f"And the provided frame, answer the following question: {user_query}"

        # Generate response using the model
        if selected_frame_pil is not None:
            result = pipe(image=selected_frame_pil, text=prompt, max_new_tokens=300)
            response = result[0]['generated_text']
        else:
            prompt_no_image = prompt.replace("and the provided frame", "")
            result = pipe(image=None, text=prompt_no_image, max_new_tokens=300)
            response = result[0]['generated_text']

    # Display the response
    print("\nAnswer:")
    print(response)
    print()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ /content/best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolo11n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 1.6s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

requirements: Ultralyt

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/50.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Device set to use cpu
Keyword argument `use_auth_token` is not a valid argument for this processor and will be ignored.
Keyword argument `image` is not a valid argument for this processor and will be ignored.



--- Generated Report ---

Based on this frame from a vehicle detection video and the following unique vehicle counts across all frames:
Car: 46
Bus: 8
Green_Auto: 1
Taxi: 9
Generate a report summarizing the unique vehicle counts and describing the scene in the selected frame. 

### Vehicle Detection Report

**Unique Vehicle Counts:**

| Vehicle Type | Count |
| --- | --- |
| Car | 46 |
| Bus | 8 |
| Green_Auto | 1 |
| Taxi | 9 |

**Scene Description:**
The scene appears to be a busy urban area with a mix of vehicles on the road. There are 46 cars, 8 buses, 1 green auto, and 9 taxis present in the scene. The high number of cars and buses suggests a high volume of traffic, while the presence of a green auto and taxis indicates a mix of personal and public transportation. The scene may be located in a city center or a major intersection. The exact location and time of day are not specified, but the high traffic volume suggests a peak hour or rush hour scenario. 

### Code
```python
# Def